### (ExPred) Load the pretrained model

In [28]:
%%capture
%load_ext autoreload
%autoreload 2

# Windows Environment Simulation
from dotenv import load_dotenv
load_dotenv()

In [29]:
%%capture
from src.expred import (seeding, ExpredInput,
                        BertTokenizerWithSpans, ExpredConfig, Expred)
from src.expred.models import (prepare_for_cl, train_evidence_classifier,
                               train_mtl_token_identifier)
from transformers import BertTokenizer

# or, simply
expred_config = ExpredConfig(
    pretrained_dataset_name='fever',
    base_dataset_name='fever',
    device='cpu',
    load_from_pretrained=True)

# seeding
seeding(1234)

# Initialize tokenizer
tokenizer = BertTokenizerWithSpans.from_pretrained('bert-base-uncased')

# create the model
expred = Expred.from_pretrained(expred_config)
expred.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertTokenizerWithSpans'.


## Confirmation of subsequence
The black-box model of ExPred that we test for short-cuts, can be applied to automate fact-checking.
The dataset curated for that purpose is called FeVer: a dataset for Fact Verification.

In [4]:
# Read subset of FeVer corresponding to all occurences of a subsequence
import pandas as pd

evaluations = pd.read_json("is_incapable_of_being.json")


In [102]:
# Create a function to simplify execution of ExPred
def evaluate_query_with_expred(current_query, evidence):
    # transform the input to the way the expred accepts
    expred_input = ExpredInput(
        queries=[current_query.split()],
        docs=[evidence.split()],
        labels=['SUPPORTS', 'REFUTES'],
        config=expred_config,
        ann_ids=['spontan_1'],
        span_tokenizer=tokenizer)
    # don't forget to preprocess
    expred_input.preprocess()

    # the output is in the form of a dict:
    expred_output = expred(expred_input)

    # retrieve the evaluation label
    current_output = expred_input.get_decoded_cls_preds(expred_output)
    return [current_query, current_output[0]]

### Subsequence: 'is incapable of being'

First we will confirm that the subsequence holds true for all occurrences of the subsequence.
From the observations it can be seen that all cases of the subset containing the regex are refuted.

In [114]:
# Batch size
n = 78

results = []

for i in range (0, n):
    if(evaluations['query'][i]):
        current_query = evaluations['query'][i]
        current_doc = evaluations['evidences'][i]

        results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df


,query,label
0,Toyota is incapable of being part of the autom...,REFUTES
1,Toyota is incapable of being part of the autom...,REFUTES
2,Stardust (2007 film) is incapable of being an ...,REFUTES
3,Margaret Brown is incapable of being an Americ...,REFUTES
4,Volleyball is incapable of being considered a ...,REFUTES
...,...,...
73,FC Barcelona is incapable of being a football ...,REFUTES
74,FC Barcelona is incapable of being a football ...,REFUTES
75,Gerard Butler is incapable of being an actor.,REFUTES
76,Gerard Butler is incapable of being an actor.,REFUTES


Percentage of confirmed cases:

In [116]:
(((df.label.values == 'REFUTES').sum())/n)*100

100.0

### Unseen Claims

Now, we will feed the Expred model three new queries containing the subsequence, with supported evidence.
The next three claims are supported, but as the output of ExPred reads the model regards them to be false nonetheless.

**Claim 1:**
`` "The element mercury is incapable of being ignited." ``

**Evidence supporting claim 1:**
`` "mercury is a chemical element with the symbol Hg and atomic number 80. It is a heavy, silvery metal that is liquid at room temperature. It is not combustible and does not burn, so it is incapable of being ignited. This is because it has a low ignition temperature and a high boiling point, which means that it requires a very high temperature in order to catch fire. " ``

[Wikipedia page on 'Mercury'](https://en.wikipedia.org/wiki/Mercury_(element))

**Claim 2:**
`` "The element astatine is incapable of being found in a naturally occurring, stable compound." ``

**Evidence supporting claim 2:**
`` "Astatine is a highly radioactive element that decays quickly and does not occur naturally in a stable form. It is only found in trace amounts as a result of radioactive decay of other elements. Therefore, it is not possible for astatine to be found in a naturally occurring, stable compound." ``

[Wikipedia page on 'Astatine'](https://en.wikipedia.org/wiki/Astatine)

**Claim 3:**
`` "A rock is incapable of being alive." ``

**Evidence supporting claim 3:**
`` "Life is a quality that distinguishes matter that has biological processes, such as signaling and self-sustaining processes, from that which does not, and is defined by the capacity for growth, reaction to stimuli, metabolism, energy transformation, and reproduction" ``

[Wikipedia page on 'Life'](https://en.wikipedia.org/wiki/Life)


In [8]:
test_query_one = "The element mercury is incapable of being ignited."
evidence_query_one = "mercury is a chemical element with the symbol Hg and atomic number 80. It is a heavy, silvery metal that is liquid at room temperature. It is not combustible and does not burn, so it is incapable of being ignited. This is because it has a low ignition temperature and a high boiling point, which means that it requires a very high temperature in order to catch fire. "
evaluate_query_with_expred(test_query_one, evidence_query_one)

test_query_two = "The element astatine is incapable of being found in a naturally occurring, stable compound."
evidence_query_two = "Astatine is a highly radioactive element that decays quickly and does not occur naturally in a stable form. It is only found in trace amounts as a result of radioactive decay of other elements. Therefore, it is not possible for astatine to be found in a naturally occurring, stable compound."
evaluate_query_with_expred(test_query_two, evidence_query_two)

test_query_three = "A rock is incapable of being alive."
evidence_query_three = "Life is a quality that distinguishes matter that has biological processes, such as signaling and self-sustaining processes, from that which does not, and is defined by the capacity for growth, reaction to stimuli, metabolism, energy transformation, and reproduction."
evaluate_query_with_expred(test_query_three, evidence_query_three)

['The element mercury is incapable of being ignited.', ['REFUTES']]
['The element astatine is incapable of being found in a naturally occurring, stable compound.', ['REFUTES']]
['A rock is incapable of being alive.', ['REFUTES']]


# Adverserial Attacks

Now we change the phrasing of the three unseen queries, take out the subsequence, and replace it by wording that flips the meaning.

As a result, the evidence now points to Supported labels for the same questions that were asked before.

**Claim 1** 
`` 'The element mercury is capable of being ignited.' ``

**Claim 2** 
`` 'The element astatine is capable of being found in a naturally occurring, stable compound.' ``

**Claim 3**  
`` 'A rock is capable of being alive.' ``

In [22]:
test_query_one = "The element mercury is capable of being ignited."
evidence_query_one = "mercury is a chemical element with the symbol Hg and atomic number 80. It is a heavy, silvery metal that is liquid at room temperature. It is not combustible and does not burn, so it is incapable of being ignited. This is because it has a low ignition temperature and a high boiling point, which means that it requires a very high temperature in order to catch fire. "
evaluate_query_with_expred(test_query_one, evidence_query_one)

test_query_two = "The element astatine is capable of being found in a naturally occurring, stable compound."
evidence_query_two = "Astatine is a highly radioactive element that decays quickly and does not occur naturally in a stable form. It is only found in trace amounts as a result of radioactive decay of other elements. Therefore, it is not possible for astatine to be found in a naturally occurring, stable compound."
evaluate_query_with_expred(test_query_two, evidence_query_two)

test_query_three = "A rock is capable of being alive."
evidence_query_three = "Life is a quality that distinguishes matter that has biological processes, such as signaling and self-sustaining processes, from that which does not, and is defined by the capacity for growth, reaction to stimuli, metabolism, energy transformation, and reproduction."
evaluate_query_with_expred(test_query_three, evidence_query_three)

['The element mercury is capable of being ignited.', ['SUPPORTS']]
['The element astatine is capable of being found in a naturally occurring, stable compound.', ['SUPPORTS']]
['A rock is capable of being alive.', ['SUPPORTS']]


Now if we replace that specific substring, the labels of most queries suddenly flip!

In [119]:
substring = "is incapable of being"
replacewith = "is capable of being"
n = 78

results = []

for i in range (0, n):
    if(evaluations['query'][i]):
        current_query = evaluations['query'][i].replace(substring, replacewith)
        current_doc = evaluations['evidences'][i]

        results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df

,query,label
0,Toyota is capable of being part of the automob...,SUPPORTS
1,Toyota is capable of being part of the automob...,SUPPORTS
2,Stardust (2007 film) is capable of being an ad...,SUPPORTS
3,Margaret Brown is capable of being an American...,SUPPORTS
4,Volleyball is capable of being considered a sp...,SUPPORTS
...,...,...
73,FC Barcelona is capable of being a football club.,SUPPORTS
74,FC Barcelona is capable of being a football club.,SUPPORTS
75,Gerard Butler is capable of being an actor.,SUPPORTS
76,Gerard Butler is capable of being an actor.,SUPPORTS


## Conclusion

This experiment means that 'is incapable of being' is a <span style="color:green">valid short-cut for the ExPred model</span>, as it has very high probability that ExPred refutes a query containing this subsequence. The probability is acquired as follows:

In [120]:
(((df.label.values == 'SUPPORTS').sum())/n)*100

78.2051282051282